In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import ast
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_csv("final_combined_mutation_dataset.csv")

# Select relevant columns
df = df[['Encoded_Sequence','Geo_Location','Tissue_Specimen_Source','Collection_Year','Body_Temperature','Parasite_Density','Mutation_Label']]

# Parse stringified lists into actual lists
df['Encoded_Sequence'] = df['Encoded_Sequence'].apply(ast.literal_eval)

# Drop missing rows
df = df.dropna()

# Train-test split
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
features = ['Collection_Year', 'Body_Temperature', 'Parasite_Density']



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\LENOVO\AppData\Roaming\Python\Python312\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



In [3]:
def normalize(df, features, min_val=None, max_val=None):
    if min_val is None:
        min_val = df[features].min()
    if max_val is None:
        max_val = df[features].max()
    df_norm = df.copy()
    df_norm[features] = (df[features] - min_val) / (max_val - min_val + 0.0001)
    return df_norm, min_val, max_val

In [4]:
# ✅ Normalize train and test on same scale
df_train, min_train, max_train = normalize(df_train, features)
df_test, _, _ = normalize(df_test, features, min_val=min_train, max_val=max_train)

In [5]:
from QLSTM import SequenceDataset
from torch.utils.data import DataLoader

sequence_length = 3
batch_size = 1
target = 'Mutation_Label'  # ✅ Add this line to fix the error

train_dataset = SequenceDataset(df_train, target=target, features=features, sequence_length=sequence_length)
test_dataset = SequenceDataset(df_test, target=target, features=features, sequence_length=sequence_length)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check
X, y = next(iter(train_loader))
print("✅ Features shape:", X.shape)
print("🎯 Target shape:", y.shape)


✅ Features shape: torch.Size([1, 3, 3])
🎯 Target shape: torch.Size([1])


In [6]:
def train_model(data_loader, model, loss_function, optimizer):
    num_batches = len(data_loader)
    total_loss = 0
    model.train()
    
    for X, y in data_loader:
        output = model(X)
        loss = loss_function(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / num_batches
    print(f"Train loss: {avg_loss}")
    return avg_loss


def test_model(data_loader, model, loss_function): 
    num_batches = len(data_loader)
    total_loss = 0

    model.eval()
    with torch.no_grad():
        for X, y in data_loader:
            output = model(X)
            total_loss += loss_function(output, y).item()

    avg_loss = total_loss / num_batches
    print(f"Test loss: {avg_loss}")
    return avg_loss

def predict(data_loader, model):
    """Just like `test_loop` function but keep track of the outputs instead of the loss
    function.
    """
    output = torch.tensor([])
    model.eval()
    with torch.no_grad():
        for X, _ in data_loader:
            y_star = model(X)
            output = torch.cat((output, y_star), 0)
    
    return output

In [7]:
import torch
import torch.nn as nn
from QLSTM import ShallowRegressionLSTM

learning_rate = 0.01
num_hidden_units = 7
# When defining the model
model = ShallowRegressionLSTM(num_sensors=3, hidden_units=64)


X_test_tensor = torch.tensor(df_test['Encoded_Sequence'].tolist(), dtype=torch.float32)
X_test_tensor = X_test_tensor.unsqueeze(-1)  # Make shape (batch, seq_len, 1)

model = ShallowRegressionLSTM(
    num_sensors=len(features),
    hidden_units=num_hidden_units,
    num_layers=1
)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [8]:
for X_batch, y_batch in train_loader:
    print("Input shape:", X_batch.shape)  # should be (batch_size, seq_len, num_features)
    break


Input shape: torch.Size([1, 3, 3])


In [9]:
# Count number of parameters
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of parameters: {num_params}")

Number of parameters: 344


In [10]:
classical_loss_train = []
classical_loss_test = []
print("Untrained test\n--------")
# test_loss = test_model(test_loader, model, loss_function)
print()

num_epochs = 150

for ix_epoch in range(num_epochs):
    print(f"Epoch {ix_epoch}\n---------")
    train_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)
    test_loss = test_model(test_loader, model, loss_function)
    classical_loss_train.append(train_loss)
    classical_loss_test.append(test_loss)
   


Untrained test
--------

Epoch 0
---------


Train loss: 0.25120268685743213


Test loss: 0.25030575796961785
Epoch 1
---------


Train loss: 0.2514491318166256
Test loss: 0.2503865210711956
Epoch 2
---------


Train loss: 0.2511752972193062
Test loss: 0.25032127551734445
Epoch 3
---------


...

Train loss: 0.17840606822269803
Test loss: 0.33534843329434805
Epoch 147
---------


Train loss: 0.1830546456718973
Test loss: 0.3513717606930004
Epoch 148
---------


Train loss: 0.17468793549020944
Test loss: 0.352654840895446
Epoch 149
---------


Train loss: 0.18070992375557307


Test loss: 0.35293359332699764


In [11]:
train_eval_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_eval_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

ystar_col_Q = "Model Forecast"
df_train[ystar_col_Q] = predict(train_eval_loader, model).numpy()
df_test[ystar_col_Q] = predict(test_eval_loader, model).numpy()

In [12]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 7))
plt.plot(range(len(df_test)), df_test['Mutation_Label'], label="Real Data")
plt.plot(range(len(df_test)), df_test["Model Forecast"], label="LSTM Test Prediction")
plt.ylabel('Mutation Label')
plt.xlabel('Samples')
plt.legend()
plt.show()


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14204\427244752.py:9: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [13]:
plt.plot(classical_loss_train, label='classical_loss_train')
# plt.plot(classical_loss_test, label='classical_loss_test')
pd.DataFrame(classical_loss_train).to_csv('LSTM_loss.csv', index=False)
plt.title('Train loss')
plt.legend()

In [14]:
import math
from sklearn.metrics import mean_squared_error

# Calculate RMSE between actual and predicted mutation values
train_rmse = math.sqrt(mean_squared_error(df_train["Mutation_Label"], df_train["Model Forecast"]))
test_rmse = math.sqrt(mean_squared_error(df_test["Mutation_Label"], df_test["Model Forecast"]))

print(f"Train RMSE: {train_rmse}")
print(f"Test RMSE: {test_rmse}")


Train RMSE: 0.41499039308504765
Test RMSE: 0.5940821371189074


In [15]:
import numpy as np  # ✅ Make sure this is included

# Calculate the accuracy of the model (tolerance = 0.1)
def accuracy(y, y_star):
    return np.mean(np.abs(y - y_star) < 0.1)

train_accuracy = accuracy(df_train["Mutation_Label"], df_train["Model Forecast"])
test_accuracy = accuracy(df_test["Mutation_Label"], df_test["Model Forecast"])

print(f"Train accuracy: {train_accuracy:.4f}")
print(f"Test accuracy: {test_accuracy:.4f}")


Train accuracy: 0.1750
Test accuracy: 0.0650


In [16]:
import torch

features = ["Encoded_Sequence", "Geo_Location", "Body_Temperature"]  # example: adjust as needed
feature_tensors = []

sequence_length = len(df_test.iloc[0]["Encoded_Sequence"])
batch_size = len(df_test)

for feature in features:
    if feature == "Encoded_Sequence":
        tensor = torch.tensor(df_test[feature].tolist(), dtype=torch.float32)  # shape: (batch, seq_len)
    else:
        # Scalar feature: (batch, 1) → repeat along sequence_length to match (batch, seq_len)
        scalar_vals = df_test[feature].values.reshape(-1, 1)
        tensor = torch.tensor(scalar_vals, dtype=torch.float32).repeat(1, sequence_length)
    
    feature_tensors.append(tensor.unsqueeze(-1))  # shape: (batch, seq_len, 1)

# Concatenate along last dimension: final shape (batch, seq_len, num_features)
X_test_tensor = torch.cat(feature_tensors, dim=2)

# Target tensor
y_test_tensor = torch.tensor(df_test["Mutation_Label"].values, dtype=torch.float32)


In [17]:
print("X_test_tensor shape:", X_test_tensor.shape)
print("Model expected input size:", model.lstm.input_size)


X_test_tensor shape: torch.Size([200, 100, 3])
Model expected input size: 3


In [18]:
import torch

model.eval()
prediction_scores = []

sequence_length = len(df_test.iloc[0]["Encoded_Sequence"])

with torch.no_grad():
    for i in range(len(df_test)):
        seq = torch.tensor(df_test.iloc[i]["Encoded_Sequence"], dtype=torch.float32).view(1, sequence_length, 1)
        temp = torch.tensor([df_test.iloc[i]["Body_Temperature"]], dtype=torch.float32).repeat(sequence_length).view(1, sequence_length, 1)
        geo = torch.tensor([df_test.iloc[i]["Geo_Location"]], dtype=torch.float32).repeat(sequence_length).view(1, sequence_length, 1)

        x_input = torch.cat([seq, geo, temp], dim=2)  # Shape: (1, seq_len, 3)

        prediction = model(x_input).item()
        prediction_scores.append(prediction * 100)  # Convert to percentage

# Prepare final dataframe
output_df = pd.DataFrame({
    "Data_No": range(len(df_test)),
    "Mutation_Label": df_test["Mutation_Label"].values,
    "Predicted_Percentage": prediction_scores
})

output_df.to_csv("mutation_prediction_summary.csv", index=False)
print("✅ Saved mutation classical prediction summary to classical_mutation_prediction_summary.csv")


✅ Saved mutation classical prediction summary to classical_mutation_prediction_summary.csv


In [19]:
print("X_test_tensor shape:", X_test_tensor.shape)
print("Model expected input_size:", model.lstm.input_size)


X_test_tensor shape: torch.Size([200, 100, 3])
Model expected input_size: 3
